modified version of the code provided by NASA's GL4HS that does NOT reduce and is intended to run locally rather than on google colab 
i also made some of my own tweaks like parallelizing downloads, some exception handling, progress bars (‼), and using more python where applicable!!

In [ ]:
import datetime
import os
import gzip
import shutil
import threading
import subprocess

start_time = datetime.datetime.now()
print("Notebook start time:", start_time.strftime('%Y-%m-%d %H:%M:%S'))

HOME_DIR = os.path.expanduser("~")
FASTQ_DIR = os.path.join(HOME_DIR, "FASTQ")
os.makedirs(FASTQ_DIR, exist_ok=True)
print(f"FASTQ directory is set to: {FASTQ_DIR}")

OSD_DATASET = '104'
GLDS_DATASET = '104'
SAMPLE_NAME = 'Mmus_C57-6J_SLS_GC_Rep1_M33'
SAMPLE_FILE_NAME = f'GLDS-{GLDS_DATASET}_rna_seq_{SAMPLE_NAME}'
FASTQ_FILE_NAME_1 = f'{SAMPLE_FILE_NAME}_R1_raw.fastq.gz'
FASTQ_FILE_NAME_2 = f'{SAMPLE_FILE_NAME}_R2_raw.fastq.gz'

with open(os.path.join(FASTQ_DIR, "KEEP_FRACTION.txt"), "w") as f:
    f.write("1.0\n")  

def download_fastq_curl(file_name):
    url = f"https://osdr.nasa.gov/geode-py/ws/studies/OSD-{OSD_DATASET}/download?source=datamanager&file={file_name}"
    output_path = os.path.join(FASTQ_DIR, file_name)
    print(f"Starting download: {file_name}")
    cmd = ["curl", "-L", "--progress-bar", "--output", output_path, url]
    subprocess.run(cmd, check=True)
    print(f"Finished download: {file_name}")

# Start downloads in parallel threads
threads = []
for fname in [FASTQ_FILE_NAME_1, FASTQ_FILE_NAME_2]:
    t = threading.Thread(target=download_fastq_curl, args=(fname,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

def decompress_fastq(input_path, output_path):
    print(f"Decompressing: {input_path}")
    with gzip.open(input_path, 'rt') as fin, open(output_path, 'w') as fout:
        for line in fin:
            fout.write(line)

def recompress_fastq(input_path):
    output_path = input_path + ".gz"
    print(f"Compressing: {input_path} → {output_path}")
    with open(input_path, 'rb') as fin, gzip.open(output_path, 'wb') as fout:
        shutil.copyfileobj(fin, fout)
    os.remove(input_path)

r1_path = os.path.join(FASTQ_DIR, FASTQ_FILE_NAME_1)
r1_decompressed = os.path.join(FASTQ_DIR, "reduced_r1.fastq")
decompress_fastq(r1_path, r1_decompressed)
if os.path.exists(r1_path):
    os.remove(r1_path)

r2_path = os.path.join(FASTQ_DIR, FASTQ_FILE_NAME_2)
r2_decompressed = os.path.join(FASTQ_DIR, "reduced_r2.fastq")
decompress_fastq(r2_path, r2_decompressed)
if os.path.exists(r2_path):
    os.remove(r2_path)

recompress_fastq(r1_decompressed)
recompress_fastq(r2_decompressed)

print("\nFinal file sizes:")
subprocess.run(f"ls -lh {FASTQ_DIR}/reduced_r*.fastq.gz", shell=True)

print("\nDisk usage:")
subprocess.run(f"du -sh {FASTQ_DIR}", shell=True)

end_time = datetime.datetime.now()
print("\nNotebook end time:", end_time.strftime('%Y-%m-%d %H:%M:%S'))
print("Total notebook time:", end_time - start_time)
